# Day8_4: ARIMA/SARIMA 예측 모델링

---

## 📚 학습 목표

**Part 1: 기초 - ARIMA 모델 이해**
1. AR(자기회귀), MA(이동평균), I(차분) 개념 이해하기
2. ARIMA(p,d,q) 모델 구조 파악하기
3. ACF/PACF 플롯으로 p, q 차수 결정하기
4. statsmodels로 ARIMA 모델 구축하고 예측하기
5. 예측 성능 평가 지표(MAE, RMSE, MAPE) 계산하기

**Part 2: 심화 - SARIMA와 자동 최적화**
1. 계절성 SARIMA(p,d,q)(P,D,Q,m) 모델 이해하기
2. pmdarima의 auto_arima로 최적 파라미터 자동 탐색하기
3. 신뢰구간과 함께 예측 결과 시각화하기
4. 실제 주가 데이터로 예측 파이프라인 구축하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| **ARIMA** | 시계열 예측의 표준 모델 | 매출 예측, 수요 예측 |
| **AR(p)** | 과거 값으로 미래 예측 | "이번 달 매출은 지난달과 비슷할 것" |
| **MA(q)** | 예측 오차 반영 | "지난달 예측이 빗나갔으니 보정" |
| **SARIMA** | 계절 패턴 반영 예측 | 여름 아이스크림, 연말 매출 |
| **auto_arima** | 최적 모델 자동 탐색 | 수백 개 조합 중 최적 선택 |
| **예측 성능 평가** | 모델 신뢰성 검증 | MAE, RMSE로 오차 측정 |

**분석가 관점**: ARIMA는 시계열 예측의 기본기로, 매출/수요/재고 예측 등 비즈니스 의사결정에 필수입니다.

---

## 🔧 환경 설정

In [2]:
!uv add pmdarima scikit-learn

Resolved 105 packages in 4ms
Audited 103 packages in 5ms


In [3]:
# 필요한 라이브러리 설치 (필요시 주석 해제)
!pip install statsmodels pmdarima yfinance plotly pandas numpy scikit-learn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 86.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 61.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 114.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 147.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 129.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 108.8 MB/s  0:00:00m0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=26ea26ec26f7eb4d036191dac3493523549f71c74861d993bfc073509ea51b2c
  Stored in directory: /home/zeus/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [pmdarima]/15 [pmdarima]ls]]


In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시계열 분석 라이브러리
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# 예측 성능 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

ModuleNotFoundError: No module named 'matplotlib'

---

# Part 1: 기초 - ARIMA 모델 이해

---

## 1.1 ARIMA 모델의 구성 요소

**ARIMA(p,d,q)** = AutoRegressive Integrated Moving Average

### AR(p) - 자기회귀 (AutoRegressive)

과거의 **값** 자체가 현재에 영향을 미친다는 개념입니다.

$$Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + \cdots + \phi_p Y_{t-p} + \epsilon_t$$

- `p`: 참고할 과거 시점의 개수 (lag)
- 예: "이번 달 매출은 지난 2개월(p=2) 매출과 관련이 있다"
- 파이는 계수! 우리가 알 순 없지만 ... 

### I(d) - 적분/차분 (Integrated)

비정상 시계열을 정상화하기 위한 **차분 횟수**입니다.

- `d=1`: 1차 차분 (추세 제거)
- `d=2`: 2차 차분 (가속도/곡률 제거)
- Day8_3에서 배운 정상성 검정 결과 활용

### MA(q) - 이동평균 (Moving Average)

과거의 **예측 오차**가 현재에 영향을 미친다는 개념입니다.

$$Y_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \cdots + \theta_q \epsilon_{t-q}$$

- `q`: 참고할 과거 예측 오차의 개수
- 예: "지난달 예측이 빗나갔으니, 그 오차를 보정해서 이번 달 예측"

### 💡 실무 비유로 이해하기

**커피숍 일일 매출 예측 시나리오**

| 요소 | 실무 해석 |
|------|----------|
| **AR(2)** | "오늘 매출은 어제, 그저께 매출과 비슷할 거야" |
| **I(1)** | "매출이 꾸준히 상승 중이니 증가분(차분)을 기준으로 보자" |
| **MA(1)** | "어제 예측이 틀렸으니, 그 오차를 반영해서 오늘 예측을 보정하자" |

**ARIMA(2,1,1)**: 과거 2일 매출 + 1차 차분 + 1일 전 오차 반영

---

## 1.2 실습 데이터: 항공 승객 수

시계열 분석의 대표적 벤치마크 데이터셋을 활용합니다.

In [ ]:
# 항공 승객 데이터 로드
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_air = pd.read_csv(url, index_col='Month', parse_dates=True)
df_air.columns = ['Passengers']

# 데이터 확인
print(f"데이터 기간: {df_air.index[0]} ~ {df_air.index[-1]}")
print(f"데이터 개수: {len(df_air)}")
df_air.head()

In [ ]:
# 시계열 시각화
fig = px.line(df_air, x=df_air.index, y='Passengers',
              title='월별 항공 승객 수 (1949-1960)')
fig.update_layout(xaxis_title='날짜', yaxis_title='승객 수')
fig.show()

### 데이터 특성 분석

그래프에서 확인할 수 있는 것:
1. **상승 추세(Trend)**: 전반적으로 승객 수 증가
2. **계절성(Seasonality)**: 매년 여름에 정점, 겨울에 저점 (12개월 주기)
3. **분산 증가**: 시간이 지날수록 변동폭이 커짐

→ **비정상 시계열**이므로 차분이 필요합니다!

---

## 1.3 ACF/PACF로 p, q 결정하기

ARIMA 모델의 핵심은 적절한 `p`, `q` 값을 찾는 것입니다.

### ACF vs PACF 해석 가이드

| 모델 | ACF 플롯 | PACF 플롯 |
|------|----------|----------|
| **AR(p)** | 점차 감소 (지수적) | p 시점 이후 급격히 절단 |
| **MA(q)** | q 시점 이후 급격히 절단 | 점차 감소 (지수적) |
| **ARMA(p,q)** | 점차 감소 | 점차 감소 |

**절단(cut-off)**: 신뢰구간(파란 영역) 안으로 급격히 떨어지는 것

In [ ]:
# 정상성을 위해 1차 차분 적용
df_diff = df_air['Passengers'].diff().dropna()

# ADF 검정
from statsmodels.tsa.stattools import adfuller

def adf_test(series, name=''):
    """ADF 검정 수행 및 결과 출력"""
    result = adfuller(series.dropna())
    print(f'>>> {name} ADF Test <<<')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    if result[1] <= 0.05:
        print("=> 정상 시계열 (p < 0.05)\n")
    else:
        print("=> 비정상 시계열 (p >= 0.05)\n")
    return result[1]

# 원본 vs 1차 차분 비교
adf_test(df_air['Passengers'], '원본')
adf_test(df_diff, '1차 차분')

In [ ]:
pip install koreanize-matplotlib
import koreanize-matplotlib


/teamspace/studios/this_studio/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ACF, PACF 플롯 그리기 (matplotlib 사용)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# ACF 플롯 - MA 차수(q) 결정에 사용
plot_acf(df_diff, lags=30, ax=axes[0], title='ACF (자기상관함수)')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')

# PACF 플롯 - AR 차수(p) 결정에 사용
plot_pacf(df_diff, lags=30, ax=axes[1], title='PACF (편자기상관함수)')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('PACF')

plt.tight_layout()
plt.show()

### ACF/PACF 해석

위 플롯에서:
- **PACF**: lag 1 이후 급격히 감소 → **p=1** 추정
- **ACF**: lag 1 이후 급격히 감소 → **q=1** 추정
- **lag 12에서 스파이크**: 12개월 주기 계절성 존재 (Part 2에서 SARIMA로 처리)

→ 비계절성 ARIMA 모델로 **ARIMA(1,1,1)** 시도

---

## 1.4 ARIMA 모델 구축 및 예측

In [ ]:
# 훈련/테스트 데이터 분할
train_size = len(df_air) - 12  # 마지막 12개월을 테스트로
train = df_air['Passengers'][:train_size]
test = df_air['Passengers'][train_size:]

print(f"훈련 데이터: {train.index[0]} ~ {train.index[-1]} ({len(train)}개)")
print(f"테스트 데이터: {test.index[0]} ~ {test.index[-1]} ({len(test)}개)")

In [ ]:
# ARIMA(1,1,1) 모델 학습
model = ARIMA(train, order=(1, 1, 1))
fitted_model = model.fit()

# 모델 요약
print(fitted_model.summary())

### 모델 요약 해석

| 항목 | 의미 |
|------|------|
| **ar.L1** | AR(1) 계수 - 이전 시점의 영향력 |
| **ma.L1** | MA(1) 계수 - 이전 오차의 영향력 |
| **sigma2** | 오차항의 분산 |
| **AIC/BIC** | 모델 적합도 (낮을수록 좋음) |
| **Ljung-Box (Q)** | 잔차 자기상관 검정 (p>0.05면 OK) |

In [ ]:
# 예측 수행
forecast = fitted_model.forecast(steps=12)

# 예측 결과 시각화
fig = go.Figure()

# 훈련 데이터
fig.add_trace(go.Scatter(
    x=train.index, y=train.values,
    mode='lines', name='훈련 데이터',
    line=dict(color='blue')
))

# 실제 테스트 데이터
fig.add_trace(go.Scatter(
    x=test.index, y=test.values,
    mode='lines', name='실제값',
    line=dict(color='green')
))

# 예측값
fig.add_trace(go.Scatter(
    x=test.index, y=forecast.values,
    mode='lines', name='예측값',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='ARIMA(1,1,1) 예측 결과',
    xaxis_title='날짜',
    yaxis_title='승객 수',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

---

## 1.5 예측 성능 평가

### 주요 평가 지표

| 지표 | 수식 | 특징 |
|------|------|------|
| **MAE** | $\frac{1}{n}\sum|y - \hat{y}|$ | 평균 절대 오차, 해석 쉬움 |
| **RMSE** | $\sqrt{\frac{1}{n}\sum(y - \hat{y})^2}$ | 큰 오차에 민감 |
| **MAPE** | $\frac{100}{n}\sum|\frac{y - \hat{y}}{y}|$ | 백분율 오차, 스케일 독립 |

In [ ]:
def evaluate_forecast(actual, predicted, name=''):
    """
    예측 성능 평가 함수
    
    Parameters:
    -----------
    actual : array-like
        실제값
    predicted : array-like
        예측값
    name : str
        모델명
    """
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    #손실값을 판단하는 함수들 (얘네들은 상대적 지표임! 다른거랑 비교해봐야함, 결국 얘네들을 줄이는 쪽으로 모델을 만들어가야함.)
    print(f"=== {name} 예측 성능 ===")
    print(f"MAE  (평균 절대 오차): {mae:.2f}")
    print(f"RMSE (평균 제곱근 오차): {rmse:.2f}")       #얘가 많이 쓰이긴 함
    print(f"MAPE (평균 백분율 오차): {mape:.2f}%")   # 얘도 많이 쓰임
    print()
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

# ARIMA(1,1,1) 성능 평가
metrics_arima = evaluate_forecast(test.values, forecast.values, 'ARIMA(1,1,1)')

NameError: name 'test' is not defined

### 💡 성능 해석 가이드

| MAPE | 해석 |
|------|------|
| < 10% | 매우 정확한 예측 |
| 10-20% | 좋은 예측 |
| 20-50% | 합리적인 예측 |
| > 50% | 부정확한 예측 |

위 결과에서 MAPE가 높다면 → **계절성을 고려하지 않았기 때문**! Part 2에서 SARIMA로 개선합니다.

---

## 1.6 다양한 ARIMA 차수 비교

In [ ]:
# 여러 ARIMA 차수 비교
orders = [
    (1, 1, 0),  # AR만
    (0, 1, 1),  # MA만
    (1, 1, 1),  # ARMA
    (2, 1, 1),  # AR(2)
    (1, 1, 2),  # MA(2)
]

results = []
for order in orders:
    try:
        model = ARIMA(train, order=order)
        fitted = model.fit()
        forecast = fitted.forecast(steps=12)
        
        mae = mean_absolute_error(test.values, forecast.values)
        rmse = np.sqrt(mean_squared_error(test.values, forecast.values))
        mape = np.mean(np.abs((test.values - forecast.values) / test.values)) * 100
        
        results.append({
            '모델': f'ARIMA{order}',
            'AIC': fitted.aic,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
        })
    except:
        pass

df_results = pd.DataFrame(results)
df_results.sort_values('AIC')

---

# Part 2: 심화 - SARIMA와 자동 최적화

---

## 2.1 SARIMA 모델 이해

**SARIMA(p,d,q)(P,D,Q,m)** = Seasonal ARIMA

기존 ARIMA에 **계절성 요소**를 추가한 모델입니다.

### 계절성 파라미터

| 파라미터 | 의미 | 예시 |
|----------|------|------|
| **P** | 계절 AR 차수 | 작년 같은 달 참조 |
| **D** | 계절 차분 횟수 | 계절성 제거 |
| **Q** | 계절 MA 차수 | 작년 예측 오차 반영 |
| **m** | 계절 주기 | 월별=12, 분기별=4 |

### 예시: SARIMA(1,1,1)(1,1,1,12)

- **(1,1,1)**: 비계절성 - AR(1), 1차 차분, MA(1)
- **(1,1,1,12)**: 계절성 - 12개월 주기, 작년 같은 달 참조

### 💡 실무 비유

**아이스크림 가게 매출 예측**

| 요소 | 해석 |
|------|------|
| **AR(1)** | "이번 달은 지난달과 비슷할 거야" |
| **Seasonal AR(1)** | "올해 8월은 작년 8월과 비슷할 거야" |
| **Seasonal D=1** | "매년 여름 성수기 패턴을 제거해서 분석하자" |

---

## 2.2 auto_arima로 최적 파라미터 탐색

SARIMA는 7개의 파라미터를 결정해야 하므로, **auto_arima**를 사용하면 편리합니다.

In [ ]:
# pmdarima 설치 (필요시)
# !pip install pmdarima

In [ ]:
import pmdarima as pm

# auto_arima로 최적 모델 탐색
sarima_model = pm.auto_arima(
    train,
    start_p=0, start_q=0,      # 시작 값
    max_p=3, max_q=3,          # 최대 값
    m=12,                       # 계절 주기 (월별)
    seasonal=True,              # 계절성 모델 사용
    d=None,                     # 차분 차수 자동 결정
    D=1,                        # 계절 차분 1회
    trace=True,                 # 탐색 과정 출력
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True               # 효율적 탐색
)

print("\n" + "="*50)
print("최적 모델:", sarima_model.summary().tables[0])

In [ ]:
# 모델 상세 요약
print(sarima_model.summary())

### auto_arima 결과 해석

- **탐색 과정**: AIC 기준으로 여러 모델 비교
- **최적 모델**: AIC가 가장 낮은 SARIMA 선택
- **Ljung-Box 검정**: 잔차 자기상관 없으면 OK (p > 0.05)
- **Jarque-Bera 검정**: 잔차 정규성 확인 (p > 0.05)

---

## 2.3 SARIMA 예측 및 신뢰구간

In [ ]:
# SARIMA 예측 (신뢰구간 포함)
n_periods = 12
forecast_sarima, conf_int = sarima_model.predict(
    n_periods=n_periods,
    return_conf_int=True  # 95% 신뢰구간 반환
)

# 예측 인덱스 생성
forecast_index = pd.date_range(
    start=train.index[-1] + pd.DateOffset(months=1),
    periods=n_periods,
    freq='MS'
)

print("예측 결과:")
pd.DataFrame({
    '예측값': forecast_sarima,
    '하한 (95%)': conf_int[:, 0],
    '상한 (95%)': conf_int[:, 1]
}, index=forecast_index)

In [ ]:
# 시각화: SARIMA 예측 + 신뢰구간
fig = go.Figure()

# 전체 실제 데이터
fig.add_trace(go.Scatter(
    x=df_air.index, y=df_air['Passengers'],
    mode='lines', name='실제 데이터',
    line=dict(color='blue')
))

# SARIMA 예측값
fig.add_trace(go.Scatter(
    x=forecast_index, y=forecast_sarima,
    mode='lines', name='SARIMA 예측',
    line=dict(color='red', dash='dash')
))

# 신뢰구간 (음영)
fig.add_trace(go.Scatter(
    x=list(forecast_index) + list(forecast_index[::-1]),
    y=list(conf_int[:, 1]) + list(conf_int[::-1, 0]),
    fill='toself',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='rgba(255,0,0,0)'),
    name='95% 신뢰구간'
))

fig.update_layout(
    title='SARIMA 예측 결과 (신뢰구간 포함)',
    xaxis_title='날짜',
    yaxis_title='승객 수',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

In [ ]:
# ARIMA vs SARIMA 성능 비교
print("=== ARIMA vs SARIMA 비교 ===")
print()

# ARIMA 성능
arima_model = ARIMA(train, order=(1, 1, 1)).fit()
arima_forecast = arima_model.forecast(steps=12)
metrics_arima = evaluate_forecast(test.values, arima_forecast.values, 'ARIMA(1,1,1)')

# SARIMA 성능
metrics_sarima = evaluate_forecast(test.values, forecast_sarima, f'SARIMA{sarima_model.order}{sarima_model.seasonal_order}')

### 결과 분석

SARIMA가 ARIMA보다 성능이 좋은 이유:
1. **계절성 반영**: 매년 비슷한 패턴을 학습
2. **적절한 파라미터**: auto_arima로 최적화
3. **분산 변화 대응**: 계절 차분으로 변동성 처리

---

## 2.4 실전: 주가 데이터 예측 파이프라인

In [ ]:
import yfinance as yf

# 삼성전자 주가 데이터 수집 (최근 2년)
ticker = "005930.KS"  # 삼성전자
df_stock = yf.download(ticker, start='2023-01-01', end='2025-01-01')

# 일별 종가 사용
stock_series = df_stock['Close'].dropna()
print(f"데이터 기간: {stock_series.index[0].date()} ~ {stock_series.index[-1].date()}")
print(f"데이터 개수: {len(stock_series)}")

# 시각화
fig = px.line(x=stock_series.index, y=stock_series.values,
              title=f'{ticker} 주가 추이')
fig.update_layout(xaxis_title='날짜', yaxis_title='종가 (KRW)')
fig.show()

In [ ]:
def stock_forecast_pipeline(series, forecast_days=30):
    """
    주가 예측 파이프라인
    
    Parameters:
    -----------
    series : pd.Series
        주가 시계열 데이터
    forecast_days : int
        예측 일수
    
    Returns:
    --------
    dict : 예측 결과 및 성능 지표
    """
    # 1. 훈련/테스트 분할
    train = series[:-forecast_days]
    test = series[-forecast_days:]
    
    print(f"훈련 데이터: {len(train)}일")
    print(f"테스트 데이터: {len(test)}일")
    print()
    
    # 2. auto_arima로 최적 모델 탐색
    print("최적 모델 탐색 중...")
    model = pm.auto_arima(
        train,
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        d=None,  # 자동 결정
        seasonal=False,  # 일별 데이터는 비계절성
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    print(f"최적 모델: ARIMA{model.order}")
    print(f"AIC: {model.aic():.2f}")
    print()
    
    # 3. 예측
    forecast, conf_int = model.predict(
        n_periods=forecast_days,
        return_conf_int=True
    )
    
    # 4. 성능 평가
    mae = mean_absolute_error(test.values, forecast)
    rmse = np.sqrt(mean_squared_error(test.values, forecast))
    mape = np.mean(np.abs((test.values - forecast) / test.values)) * 100
    
    print(f"=== 예측 성능 ===")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    
    return {
        'model': model,
        'train': train,
        'test': test,
        'forecast': forecast,
        'conf_int': conf_int,
        'metrics': {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}
    }

# 파이프라인 실행
result = stock_forecast_pipeline(stock_series, forecast_days=30)

In [ ]:
# 예측 결과 시각화
fig = go.Figure()

# 최근 90일 + 예측 30일 표시
recent_train = result['train'][-60:]

fig.add_trace(go.Scatter(
    x=recent_train.index, y=recent_train.values,
    mode='lines', name='훈련 데이터',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=result['test'].index, y=result['test'].values,
    mode='lines', name='실제값',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=result['test'].index, y=result['forecast'],
    mode='lines', name='예측값',
    line=dict(color='red', dash='dash')
))

# 신뢰구간
fig.add_trace(go.Scatter(
    x=list(result['test'].index) + list(result['test'].index[::-1]),
    y=list(result['conf_int'][:, 1]) + list(result['conf_int'][::-1, 0]),
    fill='toself',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='rgba(255,0,0,0)'),
    name='95% 신뢰구간'
))

fig.update_layout(
    title='주가 예측 결과 (ARIMA)',
    xaxis_title='날짜',
    yaxis_title='종가 (KRW)',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

### 💡 주가 예측 시 주의사항

1. **효율적 시장 가설**: 이론적으로 주가는 예측 불가 (랜덤워크)
2. **ARIMA 한계**: 외부 요인(뉴스, 경제지표) 반영 불가
3. **실무 활용**: 단기 추세 파악, 변동성 예측에 참고용
4. **투자 의사결정**: ARIMA만으로 투자 결정 금물!

**진짜 활용 분야**: 매출 예측, 수요 예측, 재고 관리 등 비즈니스 데이터

---

# 🎯 실습 퀴즈

---

## 퀴즈 1 ⭐

아래 코드를 완성하여 ARIMA(2,1,0) 모델을 학습하고 예측을 수행하세요.

```python
from statsmodels.tsa.arima.model import ARIMA

# 데이터 준비 (이미 train 변수에 훈련 데이터가 있다고 가정)
# ARIMA(2,1,0) 모델 학습
model = ARIMA(train, order=______)  # (p=2, d=1, q=0)
fitted = model.fit()

# 12개월 예측
forecast = fitted._______(steps=12)
print(forecast)
```

In [ ]:
# 퀴즈 1 풀이


## 퀴즈 2 ⭐

MAE, RMSE, MAPE 중 **스케일에 독립적**이어서 다른 데이터셋 간 비교가 가능한 지표는 무엇인가요?

아래 코드를 완성하여 해당 지표를 계산하세요.

```python
import numpy as np

actual = np.array([100, 150, 200, 180])
predicted = np.array([110, 140, 190, 170])

# 스케일 독립적 지표 계산 (백분율)
metric = np.mean(np.abs((______ - ______) / ______)) * 100
print(f"지표 값: {metric:.2f}%")
```

In [ ]:
# 퀴즈 2 풀이


## 퀴즈 3 ⭐⭐

ACF/PACF 플롯 해석에 관한 문제입니다.

PACF 플롯에서 lag 2 이후 급격히 0으로 절단되고, ACF 플롯이 점차 감소하는 패턴이 나타났습니다. 이 경우 어떤 모델이 적합한지 코드로 확인하세요.

```python
# PACF 절단: lag 2 → p = ?
# ACF 점차 감소 → 순수 AR 모델

# 적합한 모델 (q=0)
p = ______
d = 1
q = ______

print(f"적합한 모델: ARIMA({p},{d},{q})")
```

In [ ]:
# 퀴즈 3 풀이


## 퀴즈 4 ⭐⭐

항공 승객 데이터에 대해 ARIMA(1,1,1) 모델을 학습하고, 마지막 12개월을 예측한 후 RMSE를 계산하는 전체 코드를 작성하세요.

```python
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터 로드
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, index_col='Month', parse_dates=True)
df.columns = ['Passengers']
series = df['Passengers']

# 훈련/테스트 분할
train = series[:-12]
test = series[-12:]

# 모델 학습 및 예측
# 여기에 코드 작성


# RMSE 계산
# 여기에 코드 작성

print(f"RMSE: {rmse:.2f}")
```

In [ ]:
# 퀴즈 4 풀이


## 퀴즈 5 ⭐⭐

SARIMA 모델의 계절성 파라미터에 관한 문제입니다.

월별 데이터에서 매년 비슷한 패턴이 반복되는 경우, `m` 값은 얼마로 설정해야 하나요? 또한 분기별 데이터라면 `m` 값은?

```python
# 월별 데이터의 계절 주기
m_monthly = ______

# 분기별 데이터의 계절 주기
m_quarterly = ______

# 주간 데이터 (매주 같은 요일 패턴)의 계절 주기
m_weekly = ______

print(f"월별: {m_monthly}, 분기별: {m_quarterly}, 주간: {m_weekly}")
```

In [ ]:
# 퀴즈 5 풀이


## 퀴즈 6 ⭐⭐⭐

pmdarima의 auto_arima를 사용하여 항공 승객 데이터에 최적의 SARIMA 모델을 찾고, 예측 성능을 평가하세요.

요구사항:
1. 계절 주기 m=12 설정
2. 계절 차분 D=1 고정
3. AIC가 가장 낮은 모델 자동 선택
4. MAE, RMSE, MAPE 모두 출력

```python
import pmdarima as pm

# 데이터 준비 (train, test 이미 분할됨)
# auto_arima로 최적 모델 탐색
model = pm.auto_arima(
    train,
    seasonal=______,
    m=______,
    D=______,
    trace=True,
    suppress_warnings=True
)

# 예측 및 성능 평가
# 여기에 코드 작성
```

In [ ]:
# 퀴즈 6 풀이


## 퀴즈 7 ⭐⭐⭐

yfinance를 사용하여 특정 종목의 주가를 수집하고, ARIMA 예측 파이프라인을 구축하세요.

요구사항:
1. 애플(AAPL) 주식 최근 1년 데이터 수집
2. auto_arima로 최적 ARIMA 모델 탐색 (비계절성)
3. 최근 30일을 테스트 데이터로 사용
4. 예측 결과를 Plotly로 시각화 (신뢰구간 포함)

```python
import yfinance as yf
import pmdarima as pm

# 1. 데이터 수집
ticker = "______"
df = yf.download(______, period="1y")
series = df['Close'].dropna()

# 2. 훈련/테스트 분할
# 여기에 코드 작성

# 3. auto_arima 모델
# 여기에 코드 작성

# 4. 시각화
# 여기에 코드 작성
```

In [ ]:
# 퀴즈 7 풀이


## 퀴즈 8 ⭐⭐⭐⭐

여러 ARIMA 차수를 비교하여 최적 모델을 선택하는 함수를 작성하세요.

요구사항:
1. 주어진 (p,d,q) 조합 리스트에 대해 모델 학습
2. 각 모델의 AIC, MAE, RMSE 계산
3. 결과를 DataFrame으로 반환
4. AIC 기준 정렬

```python
def compare_arima_models(train, test, orders):
    """
    여러 ARIMA 차수 비교
    
    Parameters:
    -----------
    train : pd.Series
    test : pd.Series
    orders : list of tuples
        예: [(1,1,0), (1,1,1), (2,1,1)]
    
    Returns:
    --------
    pd.DataFrame
    """
    results = []
    
    for order in orders:
        try:
            # 여기에 코드 작성
            pass
        except:
            continue
    
    return pd.DataFrame(results).sort_values('AIC')

# 테스트
orders = [(1,1,0), (0,1,1), (1,1,1), (2,1,1), (1,1,2), (2,1,2)]
comparison = compare_arima_models(train, test, orders)
comparison
```

In [ ]:
# 퀴즈 8 풀이


## 퀴즈 9 ⭐⭐⭐⭐

롤링 예측(Rolling Forecast)을 구현하세요. 한 시점씩 예측하고 실제값을 훈련 데이터에 추가하는 방식입니다.

요구사항:
1. 항공 승객 데이터의 마지막 12개월을 테스트
2. 매 시점 ARIMA(1,1,1) 모델로 1개월 예측
3. 예측 후 실제값을 훈련 데이터에 추가
4. 롤링 예측 vs 일괄 예측 성능 비교

```python
def rolling_forecast(series, test_size, order):
    """
    롤링 예측 수행
    
    Parameters:
    -----------
    series : pd.Series
    test_size : int
    order : tuple
    
    Returns:
    --------
    list : 롤링 예측값
    """
    history = list(series[:-test_size])
    predictions = []
    
    for i in range(test_size):
        # 여기에 코드 작성
        # 1. 현재 history로 모델 학습
        # 2. 1시점 예측
        # 3. 실제값을 history에 추가
        pass
    
    return predictions

# 롤링 예측 실행 및 성능 비교
# 여기에 코드 작성
```

In [ ]:
# 퀴즈 9 풀이


## 퀴즈 10 ⭐⭐⭐⭐⭐

**종합 프로젝트**: 시계열 예측 파이프라인 클래스를 구현하세요.

요구사항:
1. 클래스명: `TimeSeriesForecaster`
2. 메서드:
   - `fit(series, test_size)`: 데이터 분할 및 모델 학습
   - `predict(steps)`: 예측 수행
   - `evaluate()`: 성능 평가 (MAE, RMSE, MAPE)
   - `plot()`: 예측 결과 시각화
3. auto_arima로 최적 모델 자동 선택
4. 계절성 옵션 지원

```python
class TimeSeriesForecaster:
    def __init__(self, seasonal=False, m=12):
        """
        Parameters:
        -----------
        seasonal : bool
            계절성 모델 사용 여부
        m : int
            계절 주기
        """
        self.seasonal = seasonal
        self.m = m
        self.model = None
        self.train = None
        self.test = None
        self.forecast = None
        self.conf_int = None
    
    def fit(self, series, test_size=12):
        """데이터 분할 및 모델 학습"""
        # 여기에 코드 작성
        pass
    
    def predict(self, steps=None):
        """예측 수행"""
        # 여기에 코드 작성
        pass
    
    def evaluate(self):
        """성능 평가"""
        # 여기에 코드 작성
        pass
    
    def plot(self):
        """시각화"""
        # 여기에 코드 작성
        pass

# 사용 예시
forecaster = TimeSeriesForecaster(seasonal=True, m=12)
forecaster.fit(df_air['Passengers'], test_size=12)
forecaster.predict()
metrics = forecaster.evaluate()
forecaster.plot()
```

In [ ]:
# 퀴즈 10 풀이


---

## 📊 학습 정리

---

### Part 1: 기초 핵심 요약

| 개념 | 설명 | 코드 예시 |
|------|------|----------|
| **AR(p)** | 과거 p개 값으로 현재 예측 | `order=(2,d,q)` → AR(2) |
| **I(d)** | d번 차분으로 정상화 | `order=(p,1,q)` → 1차 차분 |
| **MA(q)** | 과거 q개 오차로 보정 | `order=(p,d,2)` → MA(2) |
| **ACF** | MA 차수(q) 결정 | q 시점 이후 절단 |
| **PACF** | AR 차수(p) 결정 | p 시점 이후 절단 |
| **MAE/RMSE/MAPE** | 예측 성능 평가 | `mean_absolute_error()` |

### Part 2: 심화 핵심 요약

| 개념 | 설명 | 코드 예시 |
|------|------|----------|
| **SARIMA** | 계절성 추가 ARIMA | `seasonal_order=(P,D,Q,m)` |
| **m (계절 주기)** | 월별=12, 분기=4, 주간=7 | `m=12` |
| **auto_arima** | 최적 파라미터 자동 탐색 | `pm.auto_arima(seasonal=True)` |
| **AIC** | 모델 선택 기준 (낮을수록 좋음) | `model.aic()` |
| **신뢰구간** | 예측 불확실성 표현 | `return_conf_int=True` |

### 💡 실무 팁

1. **정상성 먼저**: Day8_3에서 배운 ADF/KPSS 검정으로 d 결정
2. **ACF/PACF 참고**: 대략적인 p, q 추정 후 auto_arima로 미세 조정
3. **계절성 확인**: 월별/분기별 데이터는 SARIMA 고려
4. **과적합 주의**: AIC/BIC로 모델 복잡도 제어
5. **롤링 예측**: 실시간 예측 시스템에서는 롤링 방식 권장
6. **신뢰구간 활용**: 예측의 불확실성도 함께 보고
7. **외부 요인**: ARIMA 한계 인식, 필요시 ARIMAX나 ML 모델 검토

---

## 다음 학습 내용

**Day8_5: 다변량 시계열 분석 (VAR + 그랜저 인과성)**
- 여러 변수 간 상호작용 분석
- 그랜저 인과성 검정으로 인과관계 파악
- VAR 모델로 다변량 예측
- 충격반응분석 기초